# Obtención del índice de movilidad por `caid` (*radius of gyration*)

## Imports

In [2]:
import sys
sys.path.append("/home/hmora/network-gen-pipeline/src")

import utils.DateUtils as du

import duckdb
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from geopandas import datasets, GeoDataFrame, read_file, GeoSeries
from pandas import concat

from shapely.geometry import shape
import shapely.wkt

import json

from h3 import h3

import matplotlib.pyplot as plt

import os
from pathlib import Path

from dotenv import load_dotenv
import os
load_dotenv()

import multiprocessing as mp

import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import pyarrow.compute as pc

duck = duckdb.connect()

/tmp/ipykernel_3206682/182862754.py:8: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


## Preparación de los datos

Dadada un fecha $d$:

In [3]:
YEAR, MONTH, DAY = ("2020", "01", "16")

Se cálcula el centroide para cada ageb en el catálogo de agebs

In [35]:
ageb_catalog = pd.read_parquet(f"{os.environ['AGEB_CATALOG']}")
ageb_catalog["geometry"] = \
            ageb_catalog["geometry"].apply(lambda x: shape(json.loads(x)))
      
ageb_catalog = GeoDataFrame(ageb_catalog, geometry='geometry', crs="EPSG:6365")

In [38]:
ageb_catalog["centroid"] = ageb_catalog["geometry"].centroid

/tmp/ipykernel_3206682/2445552291.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ageb_catalog["centroid"] = ageb_catalog["geometry"].centroid


El centroide es transformado a su representación en H3 res. 15

In [6]:
ageb_catalog["centroid"] = ageb_catalog["centroid"].apply(lambda p: h3.geo_to_h3(p.y, p.x, 15))

In [7]:
ageb_catalog

,cve_geo,cve_agee,cve_agem,cve_loc,cve_ageb,nom_agee,nom_agem,geometry,type,centroid
0,2501500010677,25,015,0001,0677,Sinaloa,Salvador Alvarado,"MULTIPOLYGON (((-108.09002 25.45957, -108.0896...",urbana,8f48031320028d0
1,080250256,08,025,None,0256,Chihuahua,None,"MULTIPOLYGON (((-107.60131 29.17987, -107.6476...",rural,8f48e654d00149d
2,0803200011456,08,032,0001,1456,Chihuahua,Hidalgo del Parral,"MULTIPOLYGON (((-105.71500 26.94922, -105.7141...",urbana,8f481e0da46eae6
3,140439037,14,043,None,9037,Jalisco,None,"MULTIPOLYGON (((-105.13341 19.57422, -105.1333...",rural,8f49ad6c1ca0280
4,0100100013986,01,001,0001,3986,Aguascalientes,Aguascalientes,"MULTIPOLYGON (((-102.24790 21.85611, -102.2468...",urbana,8f498ead97821ab
...,...,...,...,...,...,...,...,...,...,...
81642,1610200011175,16,102,0001,1175,Michoacán de Ocampo,Uruapan,"MULTIPOLYGON (((-102.02068 19.40206, -102.0209...",urbana,8f498501b420cc9
81643,1611200010385,16,112,0001,0385,Michoacán de Ocampo,Zitácuaro,"MULTIPOLYGON (((-100.34379 19.45174, -100.3434...",urbana,8f498635c026c8d
81644,1605300011183,16,053,0001,1183,Michoacán de Ocampo,Morelia,"MULTIPOLYGON (((-101.17323 19.72563, -101.1734...",urbana,8f498016d4eda72
81645,1603800010583,16,038,0001,0583,Michoacán de Ocampo,Huetamo,"MULTIPOLYGON (((-100.89441 18.62588, -100.8944...",urbana,8f498419b11ecc9


Se obtienen los pings registrados en $d$:

In [9]:
pings_table = \
    (ds.dataset(f"{os.environ['WAREHOUSE_PATH']}/pings_with_home_ageb/year={YEAR}/month={MONTH}/day={DAY}/cve_zm=09.01")) \
        .to_table().combine_chunks()
pings_table = pings_table.slice(length=1000)

En este caso se considera sólo una muestra de 1000 pings

In [11]:
pings = pings_table.to_pandas()

In [13]:
ageb_catalog = ageb_catalog.drop(columns=["geometry"])
ageb_catalog

,cve_geo,cve_agee,cve_agem,cve_loc,cve_ageb,nom_agee,nom_agem,type,centroid
0,2501500010677,25,015,0001,0677,Sinaloa,Salvador Alvarado,urbana,8f48031320028d0
1,080250256,08,025,None,0256,Chihuahua,None,rural,8f48e654d00149d
2,0803200011456,08,032,0001,1456,Chihuahua,Hidalgo del Parral,urbana,8f481e0da46eae6
3,140439037,14,043,None,9037,Jalisco,None,rural,8f49ad6c1ca0280
4,0100100013986,01,001,0001,3986,Aguascalientes,Aguascalientes,urbana,8f498ead97821ab
...,...,...,...,...,...,...,...,...,...
81642,1610200011175,16,102,0001,1175,Michoacán de Ocampo,Uruapan,urbana,8f498501b420cc9
81643,1611200010385,16,112,0001,0385,Michoacán de Ocampo,Zitácuaro,urbana,8f498635c026c8d
81644,1605300011183,16,053,0001,1183,Michoacán de Ocampo,Morelia,urbana,8f498016d4eda72
81645,1603800010583,16,038,0001,0583,Michoacán de Ocampo,Huetamo,urbana,8f498419b11ecc9


Se pega el centroide obtenido a los pings considerados

In [16]:
pings_with_home_ageb_centroid = duck.sql("""
SELECT a.*, b.centroid AS home_ageb_centroid
FROM 
    pings AS a
    INNER JOIN
    ageb_catalog AS b
    ON a.home_ageb = b.cve_geo
""").df()

In [17]:
pings_with_home_ageb_centroid

,utc_timestamp,cdmx_datetime,caid,latitude,longitude,horizontal_accuracy,cve_geo,cve_mun,h3index_15,home_ageb,home_ageb_centroid
0,1579119385,2020-01-16 02:16:25+00:00,97da803d06dfb2c11ede468e29962b828ae09b0a4b49da...,19.676581,-99.069746,16.00,1512500070054,15125,8f4995a9a9b1201,1512500070054,8f4995a9a993394
1,1579154452,2020-01-16 12:00:52+00:00,506cb252ab3d234d9d053405abcd39b5d6834d40cfa178...,19.288449,-99.001951,16.00,0901100010274,09011,8f49958e64146e6,0901000011203,8f4995848416858
2,1579186801,2020-01-16 21:00:01+00:00,10337167cc4b92ee775e23db2bafe621e4cd6090b624b9...,19.825836,-98.892894,52.20,1508400010172,15084,8f49953b309244b,1508400010346,8f49953b310bb74
3,1579181563,2020-01-16 19:32:43+00:00,bf720f02064f22f960a6610eaa0dad7465599d41463635...,19.455419,-99.210727,15.66,0901600010232,09016,8f4995ba805b26d,0900300010111,8f499584ac4d219
4,1579191460,2020-01-16 22:17:40+00:00,7e1c404c434672c7bdd15bc24d0196d78a601de81e11f0...,19.419498,-99.009565,2.20,150581842,15058,8f49958ca553383,0901000010135,8f4995ba51abb94
...,...,...,...,...,...,...,...,...,...,...,...
942,1579167824,2020-01-16 15:43:44+00:00,100f0c0b4685b14c22a7637694709318a9240d1ac92eef...,19.302173,-98.864023,25.80,1503900031692,15039,8f499589d8edb83,1503900031692,8f499589dba5206
943,1579168180,2020-01-16 15:49:40+00:00,100f0c0b4685b14c22a7637694709318a9240d1ac92eef...,19.302173,-98.864023,25.80,1503900031692,15039,8f499589d8edb83,1503900031692,8f499589dba5206
944,1579168536,2020-01-16 15:55:36+00:00,100f0c0b4685b14c22a7637694709318a9240d1ac92eef...,19.302173,-98.864023,25.80,1503900031692,15039,8f499589d8edb83,1503900031692,8f499589dba5206
945,1579168972,2020-01-16 16:02:52+00:00,100f0c0b4685b14c22a7637694709318a9240d1ac92eef...,19.302173,-98.864023,25.80,1503900031692,15039,8f499589d8edb83,1503900031692,8f499589dba5206


## Cálculo de distancias desde el `home_ageb` ($\overrightarrow{r_i}$'s)

Se obtienen los distintos traslados registrados

In [20]:
ris_table = duck.sql("""
SELECT DISTINCT caid, home_ageb_centroid, h3index_15
FROM pings_with_home_ageb_centroid
""").df()
ris_table

,caid,home_ageb_centroid,h3index_15
0,97da803d06dfb2c11ede468e29962b828ae09b0a4b49da...,8f4995a9a993394,8f4995a9a9b1201
1,506cb252ab3d234d9d053405abcd39b5d6834d40cfa178...,8f4995848416858,8f49958e64146e6
2,10337167cc4b92ee775e23db2bafe621e4cd6090b624b9...,8f49953b310bb74,8f49953b309244b
3,bf720f02064f22f960a6610eaa0dad7465599d41463635...,8f499584ac4d219,8f4995ba805b26d
4,7e1c404c434672c7bdd15bc24d0196d78a601de81e11f0...,8f4995ba51abb94,8f49958ca553383
...,...,...,...
479,21630d4db22d2df02ee003116a0e60d949d5a12b67f93e...,8f49b36c8165359,8f49b36cdc8e136
480,21630d4db22d2df02ee003116a0e60d949d5a12b67f93e...,8f49b36c8165359,8f49959804eb4d8
481,5fa894bbb5e2a8fe6793a6c8170be3f82d4970cb1a3a93...,8f49958dcaeb854,8f499585987144d
482,b766012473913bba0c08858e4f0b21762f2a00711b3cb3...,8f4995b04581ba9,8f4995b0459a011


Se calcula la distancia entre ambas celdas H3 mediante la función [h3_distance](https://github.com/uber/h3-py/blob/ae9865c1522e0b62dc668d6b2beb49cd1b4ec63f/src/h3/api/_api_template.py#L248).

> Compute the H3 distance between two cells. The H3 distance is defined as the length of the shortest path between the cells in the graph formed by connecting adjacent cells. This function will raise an exception if the cells are too far apart to compute the distance.

In [21]:
ris_table["r_i"] = ris_table[["h3index_15", "home_ageb_centroid"]] \
    .apply(lambda x : h3.h3_distance(x["h3index_15"], x["home_ageb_centroid"]), axis=1)
ris_table

,caid,home_ageb_centroid,h3index_15,r_i
0,97da803d06dfb2c11ede468e29962b828ae09b0a4b49da...,8f4995a9a993394,8f4995a9a9b1201,197
1,506cb252ab3d234d9d053405abcd39b5d6834d40cfa178...,8f4995848416858,8f49958e64146e6,21580
2,10337167cc4b92ee775e23db2bafe621e4cd6090b624b9...,8f49953b310bb74,8f49953b309244b,774
3,bf720f02064f22f960a6610eaa0dad7465599d41463635...,8f499584ac4d219,8f4995ba805b26d,13039
4,7e1c404c434672c7bdd15bc24d0196d78a601de81e11f0...,8f4995ba51abb94,8f49958ca553383,19849
...,...,...,...,...
479,21630d4db22d2df02ee003116a0e60d949d5a12b67f93e...,8f49b36c8165359,8f49b36cdc8e136,1824
480,21630d4db22d2df02ee003116a0e60d949d5a12b67f93e...,8f49b36c8165359,8f49959804eb4d8,10639
481,5fa894bbb5e2a8fe6793a6c8170be3f82d4970cb1a3a93...,8f49958dcaeb854,8f499585987144d,18647
482,b766012473913bba0c08858e4f0b21762f2a00711b3cb3...,8f4995b04581ba9,8f4995b0459a011,214


## Cálculo de centro de masa por caid ($\overrightarrow{r_{cm}}$)

In [33]:
caids_with_mass_center = duck.sql("""
WITH
/* Obtención del número de celdas distintas dentro de las cuáles se movió el caid */
locations_per_caid AS (
    SELECT caid
        , COUNT(DISTINCT h3index_15) AS n
    FROM ris_table
    GROUP BY 1
)

/* Suma de r_i's por caid */
, sum_of_ris AS (
    SELECT caid
        , SUM(r_i) AS sum_of_ris
    FROM ris_table
    GROUP BY 1
)

/* Obtención del centro de masa como la suma de las distancias desde el origen  sobre el número de
    lugares */
, caids_with_mass_center AS (
    SELECT a.caid
        , (sum_of_ris / n) AS mass_center
    FROM 
        sum_of_ris AS a
        LEFT JOIN
        locations_per_caid AS b
        ON a.caid = b.caid
)

SELECT a.caid
        , a.sum_of_ris
        , b.n
        , c.mass_center
    FROM 
        sum_of_ris AS a
        LEFT JOIN
        locations_per_caid AS b
        ON a.caid = b.caid
        LEFT JOIN
        caids_with_mass_center AS c
        ON a.caid = c.caid
""").df()
caids_with_mass_center

,caid,sum_of_ris,n,mass_center
0,97da803d06dfb2c11ede468e29962b828ae09b0a4b49da...,197.0,1,197.0
1,506cb252ab3d234d9d053405abcd39b5d6834d40cfa178...,21580.0,1,21580.0
2,10337167cc4b92ee775e23db2bafe621e4cd6090b624b9...,774.0,1,774.0
3,bf720f02064f22f960a6610eaa0dad7465599d41463635...,25936.0,2,12968.0
4,51053c3446aaaa81d77d2da047ec1566b89ff6752a6338...,23683.0,1,23683.0
...,...,...,...,...
236,bfc4cc78e42e5d61e703f2beb5a56eb220afb99aa1bc91...,10750.0,1,10750.0
237,a6643364d106ef2116f377a04c9a3702ae649fc216d2d6...,17982.0,4,4495.0
238,b50c7fad5ea055d7d09eb236e1d57099f85c4da0f30286...,1653.0,1,1653.0
239,916e16393eaca8540b30bebd070eb329124f7760dc83e2...,3135.0,1,3135.0


## Cálculo de las distancias con respecto al centro de masa $|\overrightarrow{r_{i}}\ -\ \overrightarrow{r_{cm}}|$

In [27]:
sum_of_diffs = duck.sql("""
WITH
/* Join de la tabla de r_i's con el catálogo de centros de masa */
ris_and_mass_center AS (
    SELECT 
        a.caid
        , a.r_i
        , b.mass_center
    FROM 
        ris_table AS a
        LEFT JOIN
        caids_with_mass_center AS b
        ON a.caid = b.caid
)

/* Cálculo del valor absoluto de las distancias entre cada r_i con el centro
    de masa correspondiente */
, ris_diffs AS (
    SELECT caid
        , ABS(r_i - mass_center) AS ri_diff
    FROM ris_and_mass_center
)

SELECT caid
    , SUM(ri_diff) AS ri_diff_sum
FROM ris_diffs
GROUP BY 1
""").df()
sum_of_diffs

,caid,ri_diff_sum
0,97da803d06dfb2c11ede468e29962b828ae09b0a4b49da...,0.0
1,506cb252ab3d234d9d053405abcd39b5d6834d40cfa178...,0.0
2,10337167cc4b92ee775e23db2bafe621e4cd6090b624b9...,0.0
3,bf720f02064f22f960a6610eaa0dad7465599d41463635...,142.0
4,7e1c404c434672c7bdd15bc24d0196d78a601de81e11f0...,108.0
...,...,...
236,33ccdb6ffd7b444e7606e15c8a4b16efff5415c231e5cf...,22.0
237,aea0a0f9958ac85fdfc27ed7f0dea7e0cf8465501b480c...,0.0
238,115b3b2d69aeb26a9ba668c63803650dcfbbebb9ff94f5...,0.0
239,2b4e9c5c0393470dc6e78e71a46a97d8ec4e704d0bbbac...,176.0


## Obtención del índice de movilidad (*radius of gyration*)

In [30]:
caids_rg = duck.sql("""
WITH
/* Obtención del número de celdas distintas por las que se movió el usuario */
locations_per_caid AS (
    SELECT caid
        , COUNT(DISTINCT h3index_15) AS n
    FROM ris_table
    GROUP BY 1
)

/* Cálculo del radio de giro como la raíz cuadrada de la suma de las distancias con respecto al
    centro de masa dividido sobre el número de celdas por caid */
, caids_radius_of_gyration AS (
    SELECT a.caid
        , SQRT(a.ri_diff_sum / b.n) AS rg
    FROM 
        sum_of_diffs AS a
        LEFT JOIN
        locations_per_caid AS b
        ON a.caid = b.caid
)

SELECT *
FROM caids_radius_of_gyration
""").df()
caids_rg

,caid,rg
0,97da803d06dfb2c11ede468e29962b828ae09b0a4b49da...,0.000000
1,506cb252ab3d234d9d053405abcd39b5d6834d40cfa178...,0.000000
2,10337167cc4b92ee775e23db2bafe621e4cd6090b624b9...,0.000000
3,bf720f02064f22f960a6610eaa0dad7465599d41463635...,8.426150
4,51053c3446aaaa81d77d2da047ec1566b89ff6752a6338...,0.000000
...,...,...
236,cb2f81b6737427d6e5127df37744f1e8ab2e235ca6db78...,0.000000
237,bfc4cc78e42e5d61e703f2beb5a56eb220afb99aa1bc91...,0.000000
238,a6643364d106ef2116f377a04c9a3702ae649fc216d2d6...,3.535534
239,b50c7fad5ea055d7d09eb236e1d57099f85c4da0f30286...,0.000000
